In [1]:
# import sys; sys.path.append('..') 
instance_url = 'https://www.sec.gov/Archives/edgar/data/1856028/000162828024010049/sdig-20231231_htm.xml' # 10-K report
# instance_url = 'https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/aapl-20200926_htm.xml' # Apple
# instance_url = 'https://www.sec.gov/Archives/edgar/data/1318605/000095017023033544/tsla-20230719_htm.xml' # 8-K report
# instance_url = 'https://www.sec.gov/Archives/edgar/data/1784970/000121390024106993/ea0223953-10qa1_nexttech_htm.xml' # 10-Q Amended report
# instance_url = 'https://www.sec.gov/Archives/edgar/data/3545/000000354524000128/alco-20240930_htm.xml'

uri = "bolt://localhost:7687"
username = "neo4j"
password = "Next2020#"
aura_password = "QBb3PYw5xaX9ks65bl2au__FBZkh5lpGfOkvcv35hiY"


from XBRL import *
from XBRL.Neo4jManager import Neo4jManager
# driver = GraphDatabase.driver(uri, auth=(username, password))

In [2]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

neo4j = Neo4jManager(uri, username, password)

# neo4j.clear_db()
report = process_report(instance_url, neo4j)


Created constraint for PRESENTATION_EDGE relationships
Created constraint for CALCULATION_EDGE relationships
Adding 103 DateNode nodes
Created 103 DateNode nodes
Export completed successfully
Created 102 NEXT relationships from DateNode to DateNode
Created 103 date nodes
Adding single CompanyNode node
Created 1 CompanyNode nodes
Export completed successfully
Created 103 HAS_PRICE relationships from DateNode to CompanyNode
Adding 11 AdminReportNode nodes
Created 11 AdminReportNode nodes
Export completed successfully
Created 8 HAS_SUB_REPORT relationships from AdminReportNode to AdminReportNode
Processing 10-K report for 2023-12-31
Adding single ReportNode node
Created 1 ReportNode nodes
Export completed successfully
Created 1 HAS_CATEGORY relationships from ReportNode to AdminReportNode
Created 1 REPORTED_ON relationships from DateNode to ReportNode
Created 1 FILED_BY relationships from ReportNode to CompanyNode

Concept types:
Concept: 12355
Abstract: 2598
HyperCube: 371
Dimension: 298

In [ ]:
for network in report.networks:
    print(f" Network:{network.name}, Calculation:{network.isCalculation}, Presentation:{network.isPresentation}")

In [ ]:
stop

In [ ]:
report.networks[0].__dict__

In [ ]:
count_report_hierarchy(report)

In [ ]:
stop

# ALL REPORTS

In [ ]:
# import sys; sys.path.append('..') 

instance_urls = ['https://www.sec.gov/Archives/edgar/data/1856028/000162828024010049/sdig-20231231_htm.xml', # 10-K report
'https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/aapl-20200926_htm.xml', # Apple
'https://www.sec.gov/Archives/edgar/data/1318605/000095017023033544/tsla-20230719_htm.xml', # 8-K report
'https://www.sec.gov/Archives/edgar/data/1784970/000121390024106993/ea0223953-10qa1_nexttech_htm.xml', # 10-Q Amended report
'https://www.sec.gov/Archives/edgar/data/3545/000000354524000128/alco-20240930_htm.xml'
]

# instance_urls = ['https://www.sec.gov/Archives/edgar/data/40545/000004054525000009/ge-20250123_htm.xml']
uri = "bolt://localhost:7687"
username = "neo4j"
password = "Next2020#"

from XBRL import *
from XBRL.Neo4jManager import Neo4jManager
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)


neo4j = Neo4jManager(uri, username, password)
# neo4j.clear_db()  


for instance_url in instance_urls:
    report = process_report(instance_url, neo4j)


## Unique Identifiers

### 1. Nodes

In [ ]:
print("1. Common Nodes Unique Identifiers:-")
print(f"Concept: {report.concepts[0].u_id}")
print(f"Abstract: {report.abstracts[0].u_id}") # same as concept
print(f"Period: {report.periods[0].u_id}")
print(f"Unit: {report.units[0].u_id}")
print(f"Date: {report.dates[0].id}")
print(f"Admin Report: {report.admin_reports[0].id}")

print("\n2. Company Nodes Unique Identifiers:-")
print(f"Company: {report.company.cik}")
print(f"Context: {report.contexts[0].u_id}")
print(f"Dimension: {report.taxonomy.dimensions[0].u_id}") # report.networks[0].hypercubes[0].dimensions[0].id
print(f"Member: {report.networks[4].hypercubes[0].dimensions[0].members[0].id}")


print("\n3. Report Nodes Unique Identifiers:-")
print(f"Fact: {report.facts[0].u_id}")


print("\n4. Other Nodes Unique Identifiers:-")
print(f"Hypercube: {report.networks[4].hypercubes[0].id}") # Hypercube IDs - some are Common, some are Company specific

# No IDs for Taxonomy and Networks
# print(f"Taxonomy: {report.taxonomy.__dict__}") 
# print(f"Network: {report.networks[0].__dict__}")

## Fetch Company:

In [1]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
uri = "bolt://localhost:7687"
username = "neo4j"
password = "Next2020#"
from XBRL import *
from XBRL.Neo4jManager import Neo4jManager
neo4j = Neo4jManager(uri, username, password)
lyft_cik = "0001543151"  # The CIK should match exactly what's in Neo4j
instance_url = 'https://www.sec.gov/Archives/edgar/data/1856028/000162828024010049/sdig-20231231_htm.xml' # 10-K report
instance_url = 'https://www.sec.gov/Archives/edgar/data/1318605/000095017023033544/tsla-20230719_htm.xml' # 8-K report
instance_url = 'https://www.sec.gov/Archives/edgar/data/1784970/000121390024106993/ea0223953-10qa1_nexttech_htm.xml' # 10-Q Amended report


# Retrieve the company node
def get_company_by_cik(neo4j: Neo4jManager, cik: str) -> Optional[CompanyNode]:
    try:
        with neo4j.driver.session() as session:
            result = session.run(
                "MATCH (c:Company {cik: $cik}) RETURN c", 
                {"cik": cik}
            ).single()
            
            if result:
                company_data = dict(result["c"].items())
                return CompanyNode.from_neo4j(company_data)
            return None
    except Exception as e:
        print(f"Error retrieving company with CIK {cik}: {e}")
        return None
    
company_node = get_company_by_cik(neo4j, lyft_cik)    

report = process_report(instance_url, neo4j,company_node)

Adding 104 DateNode nodes
Created 104 DateNode nodes
Export completed successfully
Created 103 NEXT relationships from DateNode to DateNode
Created 104 date nodes
Using company node: UBER TECHNOLOGIES INC (CIK: 0001543151)
Created 104 HAS_PRICE relationships from DateNode to CompanyNode
Adding 11 AdminReportNode nodes
Created 11 AdminReportNode nodes
Export completed successfully
Created 8 HAS_SUB_REPORT relationships from AdminReportNode to AdminReportNode
Processing 8-K report for 2023-07-19
Adding single ReportNode node
Created 1 ReportNode nodes
Export completed successfully
Created 1 HAS_CATEGORY relationships from ReportNode to AdminReportNode
Created 1 REPORTED_ON relationships from DateNode to ReportNode
Created 1 FILED_BY relationships from ReportNode to CompanyNode

Concept types:
Concept: 12235
Abstract: 2594
HyperCube: 371
Other: 91
LineItems: 367
Member: 2381
Dimension: 296
Guidance: 6
Built 1 unique periods
Built 0 unique units
Built 1 unique contexts
Adding 22 Concept no

In [4]:
company_node

CompanyNode(cik='0001759509', name='LYFT INC', ticker='LYFT', exchange='NAS', sector='Technology', industry='SoftwareApplication', fiscal_year_end=None, cusip=None, figi=None, class_figi=None, sic=None, sic_name=None, sector_etf='XLK', industry_etf='IGV', mkt_cap=5877954736.0, employees=2945, shares_out=406290000.0, ipo_date=None)

In [ ]:
report.report.id

### 2. Relationships

network_specific_relationships = { RelationType.PRESENTATION_EDGE, RelationType.CALCULATION_EDGE,}

Important Points: 
- The uniqueness is enforced in the MERGE clause:
A presentation/calculation relationship is considered unique if it has the same:
- cik
- report_id
- network_name
- parent_id
- child_id
- parent_level (context_id for Calculation)
- child_level  (context_id for Presentation)

- Even if other properties differ (like weights, levels, orders), Neo4j will update the existing relationship rather than create a duplicate if these identifying properties match.

-This ensures that each parent-child relationship within a specific network for a specific company exists only once. 

-In calculation networks, both parent and child facts must share the same context for the calculation to be valid - context includes dimensional qualifiers, it will differentiate between same source-target relationships that have different dimensional contexts, even within the same time period.

### for presentation edges

    unique_key = (
        cik,                # Company identity
        report_id,          # Specific filing
        network_name,       # Network context
        parent_id,          # Parent concept
        child_id,          # Child concept
        parent_level,       # Position in hierarchy
        child_level        # Position in hierarchy
    )


In [ ]:
pres_df = report.neo4j.fetch_relationships(RelationType.PRESENTATION_EDGE)
pres_df.head(1)


### for calculation edges

    unique_key = (
        cik,                # Company identity
        report_id,          # Specific filing
        network_name,       # Network context
        parent_id,          # Parent concept
        child_id,          # Child concept
        context_id         # Full context including dimensions
    )

In [ ]:
calc_df = report.neo4j.fetch_relationships(RelationType.CALCULATION_EDGE)
calc_df.head(1)


In [ ]:
calc_df.groupby(['parent_qname']).count()

In [ ]:
calc_df.groupby(['parent_qname', 'context_id']).count()


In [ ]:
report.report.instanceFile


In [ ]:
print(pres_df.columns.tolist())
print(calc_df.columns.tolist()) 


In [ ]:
report.networks[0].network_uri.split('/')[-1],report.networks[0].network_uri

In [ ]:
for fact in report.model_xbrl.factsInInstance:
    print(fact.qname, fact.context.id)
    break


In [ ]:
report.networks[0].presentation

In [ ]:
5. **Context Node:** cik_perioduid_dimensionuids_memberuids (3545_duration_2023-10-01_2024-10-01_dim1_dim2_mem1_mem2)
5. **Context Node:** cik_perioduid_dim1_dim2_mem1_mem2 (0000012345_duration_2023-10-01_2024-10-01_http://fasb.org/us-gaap/2023:Axis1_http://fasb.org/us-gaap/2023:Axis2_http://fasb.org/us-gaap/2023:Member1_http://fasb.org/us-gaap/2023:Member2)

In [ ]:
stop

In [72]:
from arelle import Cntlr, XbrlConst
from arelle.ModelXbrl import ModelXbrl

def get_all_footnotes(model_xbrl: ModelXbrl):
    if model_xbrl is None:
        print("No valid XBRL model provided")
        return []
        
    footnotes = []
    
    try:
        # 1. Get fact-footnote relationships
        print("Checking fact-footnote relationships...")
        fact_footnote_rels = model_xbrl.relationshipSet(XbrlConst.factFootnote)
        if fact_footnote_rels:
            for rel in fact_footnote_rels.modelRelationships:
                if rel.fromModelObject is not None and rel.toModelObject is not None:
                    footnotes.append({
                        'type': 'fact-footnote',
                        'fact_id': getattr(rel.fromModelObject, 'id', 'N/A'),
                        'fact_value': getattr(rel.fromModelObject, 'value', 'N/A'),
                        'fact_concept': getattr(rel.fromModelObject, 'qname', 'N/A'),
                        'footnote_text': getattr(rel.toModelObject, 'textValue', 'N/A'),
                        'footnote_lang': getattr(rel.toModelObject, 'xmlLang', 'N/A'),
                        'footnote_role': getattr(rel.toModelObject, 'role', 'N/A')
                    })

        # 2. Get fact-explanatoryFact relationships
        print("Checking fact-explanatoryFact relationships...")
        explanatory_rels = model_xbrl.relationshipSet(XbrlConst.factExplanatoryFact)
        if explanatory_rels:
            for rel in explanatory_rels.modelRelationships:
                if rel.fromModelObject is not None and rel.toModelObject is not None:
                    footnotes.append({
                        'type': 'explanatory-fact',
                        'fact_id': getattr(rel.fromModelObject, 'id', 'N/A'),
                        'fact_value': getattr(rel.fromModelObject, 'value', 'N/A'),
                        'fact_concept': getattr(rel.fromModelObject, 'qname', 'N/A'),
                        'explanation_text': getattr(rel.toModelObject, 'value', 'N/A'),
                        'explanation_concept': getattr(rel.toModelObject, 'qname', 'N/A')
                    })

        # Debug information
        print("\nDebug Information:")
        print(f"Total facts in document: {len(model_xbrl.facts)}")
        print("Available relationship sets:")
        for relSet in model_xbrl.relationshipSets.keys():
            print(f"- {relSet}")
        
    except Exception as e:
        print(f"Error processing footnotes: {str(e)}")
    
    return footnotes

controller = Cntlr.Cntlr(logFileName="logfile.txt")
model_xbrl = controller.modelManager.load(url)


def test_footnotes():
    urls = [
        "https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/aapl-20200926_htm.xml",
        "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231_htm.xml"
    ]
    
    for url in urls:
        print(f"\nTesting URL: {url}")
        controller = Cntlr.Cntlr(logFileName="logfile.txt")
        model_xbrl = controller.modelManager.load(url)
        
        if model_xbrl is not None:
            print("Document loaded successfully")
            footnotes = get_all_footnotes(model_xbrl)
            controller.modelManager.close()
        else:
            print("Failed to load document")

In [61]:
from arelle import XbrlConst
from arelle.ModelXbrl import ModelXbrl

def get_footnotes(model_xbrl: ModelXbrl):
    # Get the relationship set for fact-footnote relationships
    fact_footnote_relationships = model_xbrl.relationshipSet(
        XbrlConst.factFootnote  # 'http://www.xbrl.org/2003/arcrole/fact-footnote'
    )
    
    # Get the relationship set for fact-explanatoryFact relationships
    explanatory_fact_relationships = model_xbrl.relationshipSet(
        XbrlConst.factExplanatoryFact  # 'http://www.xbrl.org/2009/arcrole/fact-explanatoryFact'
    )
    
    footnotes = []
    
    # Process fact footnotes
    if fact_footnote_relationships:
        for rel in fact_footnote_relationships.modelRelationships:
            footnotes.append({
                'type': 'fact-footnote',
                'from_fact': rel.fromModelObject,
                'footnote': rel.toModelObject,
                'role': rel.roleURI  # Usually 'http://www.xbrl.org/2003/role/footnote'
            })
    
    # Process explanatory footnotes
    if explanatory_fact_relationships:
        for rel in explanatory_fact_relationships.modelRelationships:
            footnotes.append({
                'type': 'explanatory-fact',
                'from_fact': rel.fromModelObject,
                'explanation': rel.toModelObject
            })
    
    return footnotes

In [70]:
from arelle import Cntlr
from arelle.ModelXbrl import load
controller = Cntlr.Cntlr(logFileName="logfile.txt")
url = 'https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/aapl-20200926_htm.xml'
model_xbrl = controller.modelManager.load(url)


footnotes = get_footnotes(model_xbrl)

# Print the footnotes
for footnote in footnotes:
    if footnote['type'] == 'fact-footnote':
        print(f"Fact: {footnote['from_fact'].value}")
        print(f"Footnote: {footnote['footnote'].value}")
        print(f"Role: {footnote['role']}")
    else:
        print(f"Fact: {footnote['from_fact'].value}")
        print(f"Explanation: {footnote['explanation'].value}")
    print("---")

# REFERENCES AND DOCUMENTATION

In [ ]:
def fetch_concept_references_and_documentation(instance_url):
    from arelle import Cntlr
    from arelle.ModelXbrl import load

    controller = Cntlr.Cntlr(logFileName="logfile.txt")
    model_xbrl = controller.modelManager.load(instance_url)

    ref_rel_set = model_xbrl.relationshipSet("http://www.xbrl.org/2003/arcrole/concept-reference")
    doc_rel_set = model_xbrl.relationshipSet("http://www.xbrl.org/2003/arcrole/concept-documentation")

    references, docs = [], []

    for concept in model_xbrl.qnameConcepts.values():
        # References
        for ref_rel in ref_rel_set.fromModelObject(concept):
            ref_obj = ref_rel.toModelObject
            if ref_obj is not None:
                parts = []
                for child in ref_obj.iterchildren():
                    parts.append(child.stringValue)  # Corrected to access as attribute
                references.append((concept.qname, " ".join(parts)))

        # Documentation
        for doc_rel in doc_rel_set.fromModelObject(concept):
            doc_obj = doc_rel.toModelObject
            if doc_obj is not None:
                parts = []
                for child in doc_obj.iterchildren():
                    parts.append(child.stringValue)  # Corrected to access as attribute
                docs.append((concept.qname, " ".join(parts)))

    return references, docs


references, docs = fetch_concept_references_and_documentation(instance_url)
print("References:", references)
print("Documentation:", docs)

references

In [ ]:
def fetch_references(instance_url):
    from arelle import Cntlr, XbrlConst
    from arelle.ModelXbrl import load

    controller = Cntlr.Cntlr(logFileName="logfile.txt")
    model_xbrl = controller.modelManager.load(instance_url)

    ref_rel_set = model_xbrl.relationshipSet("http://www.xbrl.org/2003/arcrole/concept-reference")
    references = []

    for concept in model_xbrl.qnameConcepts.values():
        for ref_rel in ref_rel_set.fromModelObject(concept):
            ref_obj = ref_rel.toModelObject
            if ref_obj:
                references.append((concept.qname, ref_obj.viewText()))

    return references


fetch_references(instance_url)

In [ ]:
def load_xbrl_and_extract_facts(instance_url):
    from arelle import Cntlr, XbrlConst
    from arelle.ModelXbrl import load
    
    controller = Cntlr.Cntlr(logFileName="logfile.txt")
    model_xbrl = controller.modelManager.load(instance_url)

    footnotes = []
    explanatory_facts = []

    footnote_rel_set = model_xbrl.relationshipSet(XbrlConst.factFootnote)
    for fact in model_xbrl.facts:
        # Gather footnotes via relationshipSet
        for footnote_rel in footnote_rel_set.fromModelObject(fact):
            note = footnote_rel.toModelObject
            if note is not None:
                footnotes.append((fact.qname, fact.value, note.stringValue))

        # Check if "explanatoryFact" is in the fact name
        if "explanatoryfact" in fact.qname.localName.lower():
            explanatory_facts.append((fact.qname, fact.value))

    return footnotes, explanatory_facts


# Load and print footnotes and explanatory facts for both instances
footnotes1, explanatory_facts1 = load_xbrl_and_extract_facts(instance_url)
print("Footnotes instance results:")
print("Footnotes:", footnotes1)
print("Explanatory facts:", explanatory_facts1)
print("\n")

footnotes2, explanatory_facts2 = load_xbrl_and_extract_facts(instance_url) 
print("Explanatory facts instance results:")
print("Footnotes:", footnotes2)
print("Explanatory facts:", explanatory_facts2)
    



In [ ]:
report.networks[0].calculation.__dict__

In [ ]:
report.networks[0].presentation.__dict__

In [ ]:
report.contexts[0].__dict__

In [ ]:
for ctx in report.contexts:
    if ctx.context_id == 'c2':
        for prop in ctx.properties:
            print(prop,":", ctx.properties[prop])
        break

In [ ]:
for ctx in report.contexts:
    if ctx.context_id == 'c5':
        for prop in ctx.properties:
            print(prop,":", ctx.properties[prop])
        break

In [ ]:
report.facts[0].__dict__

In [ ]:
report.networks[0].network_uri

In [ ]:
for node in report.networks[0].calculation.nodes.values():
    print(node)
    for child in node.children:
        print(child)
    break

In [ ]:
report.pure_abstracts[:2]


In [ ]:
len(report.pure_abstracts)

In [ ]:
len(report.abstracts)

In [ ]:
len(report._abstract_lookup)

In [ ]:
# for abstract in report.abstracts:
#     print(abstract.category)
from collections import Counter
category_counts = Counter(abstract.category for abstract in report.abstracts)
for category, count in category_counts.items():
    print(f"{category}: {count}")


In [ ]:
print("\nReport → Networks → Categories")
network_categories = {}
for network in report.networks:
    network_categories[network.category] = network_categories.get(network.category, 0) + 1
for category, count in sorted(network_categories.items()):
    print(f"├→ {category}: {count}")
print(f"└→ Total Categories: {len(network_categories)}")

In [ ]:
report.networks[0].presentation.nodes.values()

In [ ]:
report.networks[0].presentation.nodes.values()


In [ ]:
from collections import defaultdict

concept_to_networks = defaultdict(list)

# Build a dictionary of concepts to the networks they appear in
for network in report.networks:
    if network.presentation and network.presentation.nodes:  # Check if presentation and nodes exist
        for concept, _ in network.presentation.nodes.items():
            concept_to_networks[concept].append(network.name)

# Find and display the first concept that appears in more than one network
for concept, networks in concept_to_networks.items():
    if len(networks) > 1:
        print(f"Concept: {concept}")
        print(f"Networks: {networks[0]} and {networks[1]}")
        print("*"*100)
    


In [ ]:
# First, validate all facts across networks
validated_facts_by_network = report.validate_all_facts()

In [ ]:
for network in report.networks:
    if network.network_uri == 'http://wetradegroup.net/role/ConsolidatedBalanceSheet':
        for node in network.presentation.nodes.values():
            print(node)


In [ ]:
for fact in validated_facts_by_network['http://wetradegroup.net/role/ConsolidatedBalanceSheet']:
    print(fact.model_fact.qname, fact.model_fact.value, fact.period.u_id, fact.model_fact.context.id)


In [14]:
# instance_url = 'https://www.sec.gov/Archives/edgar/data/3545/000000354524000128/alco-20240930_htm.xml'
# from arelle import Cntlr
# from arelle.ModelXbrl import load
# controller = Cntlr.Cntlr(logFileName="logfile.txt")
# model_xbrl = controller.modelManager.load(instance_url)



In [ ]:
validated_facts_by_network.keys()

In [ ]:
stop

In [ ]:
for network in report.networks:
    for hypercube in network.hypercubes:
        print(network.name, hypercube.qname, len(hypercube.dimensions))
        # if hypercube.dimensions:
        #     print(hypercube.dimensions[0].qname)


In [ ]:
node.concept.facts

In [ ]:
def get_hypercube_dimensions(network):
    hypercube_dimensions = []
    for hypercube in network.hypercubes:
        print("     HYPERCUBE:", hypercube.qname, ", TotalDimensions:", len(hypercube.dimensions))
        print(f"                closed:{hypercube.closed} all:{hypercube.is_all}")
        print(f"                concepts: {[concept.qname for concept in hypercube.concepts]}")
        for i, dim in enumerate(hypercube.dimensions):
            print(f"         DIMENSION{i+1}: {dim.qname}")
            hypercube_dimensions.append(dim.qname)
    return hypercube_dimensions


for i, network in enumerate(report.networks):
    print(f"\nNETWORK{i+1}: {network.name}")
    hypercube_dimensions = get_hypercube_dimensions(network)

    j = 0
    for node in getattr(network.presentation, 'nodes', {}).values():
        # print("     NODE:", node.concept.qname)
        if node.concept and node.concept.__class__ != AbstractConcept:
            j += 1
            print(f"     CONCEPT {j}:", node.concept.qname)
            print(f"Total Facts in Concept: {len(node.concept.facts)}")
            for i,fact in enumerate(node.concept.facts):                
                print(f"         FACT{i+1}: {fact.qname}, {fact.period.u_id}")
                for i, (dim, member) in enumerate(fact.dims_members):
                    print(f"           DIMENSION{i+1}: {dim.qname}")

    print("-"*100)    
    break


In [ ]:
for fact in report.facts:
    # if len(fact.dims_members) > 1:
    print(fact.u_id)
    for dim, member in fact.dims_members:
        print(dim.qname, member.qname)

In [ ]:
rough = [(1,2), (3,4), (5, None)]

for dim, member in rough:
    if member is None:
        print(dim)


In [7]:
for fact in report.facts:
    for dim, member in fact.dims_members:
        if member is None:
            print(fact.u_id, dim.qname)


In [ ]:
for dim, member in report.facts[0].dims_members:
    print(dim.default_member.qname)


In [7]:
def print_presentation_hierarchy(presentation: Presentation) -> None:
    def recursive_print(node: PresentationNode, indent: int = 0):
        print("  " * indent + f"{node.concept.qname} ({node.order})")
        for child_id in sorted(node.children, key=lambda cid: presentation.nodes[cid].order):
            recursive_print(presentation.nodes[child_id], indent + 1)

    # Start printing from the root nodes
    for root in sorted(presentation.roots, key=lambda n: n.order):
        recursive_print(root)


In [ ]:
print_presentation_hierarchy(report.networks[0].presentation)


In [ ]:
report.networks[0].presentation.nodes

In [ ]:
report.net

In [ ]:
report.company

In [ ]:
report.report

In [ ]:
stop

In [ ]:
count_report_hierarchy(report) 

In [ ]:
report.taxonomy.dimensions[12].__dict__

In [ ]:
report.facts[14].__dict__

In [ ]:
for fact in report.facts:
    if fact.dims_members:
        print([(dim_concept.qname, dim_member.qname) for dim_concept, dim_member in fact.dims_members])
        break

In [7]:
for dim_member in report.facts[14].dims_members:
    print(dim_member[0].qname)
    for member in dim_member[0].members_dict.values():
        print(member.qname)


In [ ]:
report.taxonomy.dimensions[12].members_dict['alco:TropicanaMember'].item.qname.localName

In [ ]:
report.networks[0]

In [ ]:
report.report

In [ ]:
j = 1
for i, dim in enumerate(report.taxonomy.dimensions):
    if len(dim.members_dict) > 1:
        print(j, dim.qname, [member.qname for member in dim.members_dict.values()])
        j += 1
        


In [ ]:
report.taxonomy.dimensions[12].__dict__

In [ ]:
report.report.periodEnd

In [ ]:
stop

In [ ]:
report.neo4j.get_neo4j_db_counts() 

In [ ]:
count_report_hierarchy(report)

In [ ]:
report.taxonomy.dimensions[1].__dict__

In [ ]:
report.taxonomy.__dict__.keys()


In [ ]:
report.taxonomy._dimension_lookup

In [ ]:
report.taxonomy.dimensions[12].__dict__

In [ ]:
# Print summary for all dimensions
for i, dimension in enumerate(report.taxonomy.dimensions):
    print(f"\nDimension: {i}. {dimension.qname}")
    print(f"Domain: {dimension.domain.qname if dimension.domain else 'No domain'}")
    print(f"Member count: {len(dimension.members_dict)}")
    if dimension.members_dict:
        print("Members:")
        for member in dimension.members_dict.values():
            print(f"  {member.qname} (Parent: {member.parent_qname})")

# Fetch Nodes/Rels from Neo4j

In [75]:
def fetch_presentation_network_with_period(uri: str, username: str, password: str):
    from neo4j import GraphDatabase
    import pandas as pd

    def run_query(tx):
        query = """
        MATCH path = (n)-[r:PRESENTATION_EDGE*1..3 {network_name: 'Condensed Consolidated Balance Sheets (Unaudited)'}]->(m)
        MATCH (m)-[hp:HAS_PERIOD]->(p:Period)  // Follow HAS_PERIOD relationship from the target node
        RETURN path, p.id as period_id, p.u_id as period_u_id
        """
        return list(tx.run(query))

    try:
        driver = GraphDatabase.driver(uri, auth=(username, password))
        with driver.session() as session:
            paths = session.execute_read(run_query)
            
            rows = []
            for record in paths:
                path = record['path']
                period_id = record.get('period_id', None)
                period_u_id = record.get('period_u_id', None)
                
                for rel in path.relationships:
                    source = path.start_node
                    target = path.end_node
                    
                    row = {
                        # Existing fields...
                        'source_type': 'Fact' if 'Fact' in source.labels else 'Abstract',
                        'source_qname': source.get('qname', ''),
                        'source_value': source.get('value', ''),
                        'source_label': source.get('label', ''),
                        'source_category': source.get('category', ''),
                        
                        'child_level': rel.get('child_level', ''),
                        'child_order': rel.get('child_order', ''),
                        'parent_level': rel.get('parent_level', ''),
                        'parent_order': rel.get('parent_order', ''),
                        'network_name': rel.get('network_name', ''),
                        'network_uri': rel.get('network_uri', ''),
                        
                        'target_type': 'Fact' if 'Fact' in target.labels else 'Abstract',
                        'target_qname': target.get('qname', ''),
                        'target_value': target.get('value', ''),
                        'target_label': target.get('label', ''),
                        'target_category': target.get('category', ''),

                        # Period information
                        'period_id': period_id,
                        'period_u_id': period_u_id
                    }
                    rows.append(row)
            
            df = pd.DataFrame(rows)
            return df
            
    finally:
        driver.close()

In [ ]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "Next2020#"


df = fetch_presentation_network_with_period(uri, username, password)
df = df[df['target_value'].notna() & (df['target_value'] != '')]
df.drop_duplicates(subset=['target_qname','target_value', 'period_id'], keep='first', inplace=True)
df = df.sort_values(by=['child_level', 'child_order'], ascending=[True, True])
df = df[['target_qname','target_value','period_id']]
print(df.period_id.value_counts())
df = df[df['period_id'] == 'instant_2024-10-01']
df




In [ ]:
df.columns

### members and hierarchies for a specific dimension that has members

In [ ]:
report.concepts[0].__dict__

In [ ]:
# Example for ProductOrServiceAxis which has 153 members
dimension = next(d for d in report.taxonomy.dimensions if d.qname == 'srt:ProductOrServiceAxis')

# Print hierarchy
def print_hierarchy(dimension):
    print(f"\nDimension: {dimension.qname}")
    print(f"Domain: {dimension.domain.qname if dimension.domain else 'No domain'}")
    
    # Get root members (those without parents)
    root_members = [m for m in dimension.members_dict.values() if not m.parent_qname]
    
    def print_tree(member, level=0):
        indent = "  " * level
        print(f"{indent}└─ {member.qname}")
        # Get children
        children = [m for m in dimension.members_dict.values() 
                   if m.parent_qname == str(member.qname)]
        for child in children:
            print_tree(child, level + 1)
    
    for root in root_members:
        print_tree(root)

print_hierarchy(dimension)

In [ ]:
for network in report.networks:
    if network.network_uri == 'http://xbrl.sec.gov/ecd/role/PvpAdjustmentsToCompensation':
        print(network.hypercubes[0].dimensions[0])

In [ ]:
stop

In [ ]:
report.networks[4].hypercubes[0].dimensions[0].member_hierarchy

In [ ]:
def compare_network_taxonomy_dimensions(report):
    """Compare dimensions between network-specific (hypercubes) and taxonomy-wide contexts"""
    
    # Get taxonomy-wide dimensions
    tax_dimensions = {d.qname: d for d in report.taxonomy.dimensions}
    
    # Get network dimensions from hypercubes in definition networks
    network_dimensions = {}
    for network in report.networks:
        if network.isDefinition:  # Using the correct property
            for hypercube in network.hypercubes:
                for dimension in hypercube.dimensions:
                    if dimension.qname not in network_dimensions:
                        network_dimensions[dimension.qname] = []
                    network_dimensions[dimension.qname].append({
                        'network': network.network_uri,
                        'hypercube': hypercube.qname,
                        'members': dimension.members_dict if hasattr(dimension, 'members_dict') else {}
                    })
    
    # Compare and print results
    print("\nDimension Comparison Analysis:")
    print("=" * 80)
    
    all_dimensions = set(tax_dimensions.keys()) | set(network_dimensions.keys())
    
    for dim_qname in sorted(all_dimensions):
        print(f"\nDimension: {dim_qname}")
        
        # Taxonomy-wide info
        if dim_qname in tax_dimensions:
            tax_dim = tax_dimensions[dim_qname]
            tax_members = len(tax_dim.members_dict) if hasattr(tax_dim, 'members_dict') else 0
            print(f"Taxonomy-wide members: {tax_members}")
            if tax_members > 0:
                print("Member QNames:")
                for member in tax_dim.members_dict.values():
                    print(f"  - {member.qname}")
        else:
            print("Not present in taxonomy-wide context")
            
        # Network-specific info
        if dim_qname in network_dimensions:
            print("\nNetwork/Hypercube appearances:")
            for network_info in network_dimensions[dim_qname]:
                member_count = len(network_info['members'])
                print(f"\nNetwork: {network_info['network']}")
                print(f"Hypercube: {network_info['hypercube']}")
                print(f"Network-specific members: {member_count}")
                if member_count > 0:
                    print("Member QNames:")
                    for member in network_info['members'].values():
                        print(f"  - {member.qname}")
        else:
            print("\nNot present in any network hypercubes")
            
        print("-" * 40)

# Use it
compare_network_taxonomy_dimensions(report)

In [ ]:
# Fetch u_id of Dimension, Domain, Member for each fact
for fact in report.facts:
    if fact.dims_members:
        print(fact.u_id, [(dim_concept.qname, dim_member.qname) for dim_concept, dim_member in fact.dims_members])
        break



In [ ]:

dimension = report.taxonomy.dimensions[1]

# 1. Get all members for this dimension
members = dimension.members_dict.values()
members


In [ ]:
report.networks[4].hypercubes[0].dimensions[0].member_hierarchy

In [ ]:
# Check if _build_members() is being called
dimension = next(d for d in report.taxonomy.dimensions if d.qname == 'srt:StatementScenarioAxis')
print("Before building members:", len(dimension.members_dict))
dimension._build_members()
print("After building members:", len(dimension.members_dict))

# Check relationship sets
dim_dom_rel_set = dimension.model_xbrl.relationshipSet(XbrlConst.dimensionDomain)
dom_mem_rel_set = dimension.model_xbrl.relationshipSet(XbrlConst.domainMember)
print("\nDimension-domain relationships exist:", bool(dim_dom_rel_set))
print("Domain-member relationships exist:", bool(dom_mem_rel_set))

# If relationships exist, check specific dimension
if dim_dom_rel_set:
    for rel in dim_dom_rel_set.fromModelObject(dimension.item):
        print(f"\nFound domain: {rel.toModelObject.qname}")

In [ ]:
report.networks[4].hypercubes[0].dimensions[0].members[0]

In [ ]:
report.taxonomy.dimensions[1].member_hierarchy


In [ ]:
report.concepts[0]

In [34]:
# Get all members
all_members = report.taxonomy.get_all_members()

# Get members for specific dimension
major_customer_members = report.taxonomy.get_dimension_members('srt:MajorCustomersAxis')


In [ ]:
major_customer_members

In [ ]:
all_members


In [ ]:
STOP

In [ ]:
for network in report.networks:
    for hypercube in network.hypercubes:
        for dim in hypercube.dimensions:
            print(len(dim.members))


In [ ]:
print("\nDimension Member Counts (Taxonomy vs Network Usage):")
print("-" * 50)
for dim in report.taxonomy.dimensions:
    print(f"{dim.name}:")
    tax_count = len(dim.members_dict) if dim.members_dict else 0
    net_counts = []
    for network in report.networks:
        for hypercube in network.hypercubes:
            for hc_dim in hypercube.dimensions:
                if hc_dim.name == dim.name:
                    net_counts.append(len(hc_dim.members))
    print(f"Taxonomy: {tax_count} | Network: {net_counts if net_counts else 0}")

In [ ]:
stop

In [ ]:
for i,fact in enumerate(report.facts):
    if fact.dims_members:
        for dim_concept, dim_member in fact.dims_members:
            print(i+1, dim_concept, dim_member)        



In [ ]:
report.networks[4].hypercubes[0].dimensions[0].member_hierarchy

In [ ]:
report.networks[2].validate_facts()


In [ ]:
for ctxt in report.model_xbrl.contextsInUse:
    print(f"\nContext ID: {ctxt.id}")

    # Extract period details
    if ctxt.isInstantPeriod:
        print(f"Period Type: Instant")
        print(f"Date: {ctxt.instantDatetime}")
    elif ctxt.isStartEndPeriod:
        print(f"Period Type: Duration")
        print(f"Start Date: {ctxt.startDatetime}")
        print(f"End Date: {ctxt.endDatetime}")
    else:
        print("Period Type: Unknown")

    # Extract all dimensions and members
    if ctxt.hasSegment or ctxt.hasScenario:
        dims = ctxt.qnameDims.items()
        if dims:
            print("Dimensions and Members:")
            for dim_qname, dim in dims:
                print(f" - Dimension QName: {dim_qname}")
                if dim.isExplicit:
                    print(f"   Member QName: {dim.memberQname}")
                elif dim.isTyped:
                    print(f"   Typed Member: {dim.typedMember}")
        else:
            print("No dimensions found.")
    else:
        print("No dimensions in this context.")


In [ ]:
report.facts[3].dims_members

In [ ]:
for fact in report.model_xbrl.factsInInstance:
    print(fact)
    break   


In [ ]:

def _extract_text(value):
    return re.sub('<[^>]+>', '', html.unescape(value)) if '<' in value and '>' in value else value

# Specify the target dimension QName
target_dimension_qname = "srt:ProductOrServiceAxis"

# Iterate through all facts in the XBRL instance
for fact in report.model_xbrl.factsInInstance:
    if not hasattr(fact, "context") or fact.context is None:
        continue  # Skip if no context is associated with the fact

    # Check if the context has dimensions
    if hasattr(fact.context, "qnameDims"):
        print(f"\nFact: {fact}")
        print("Dimensions found in this context:")

        # List all dimensions in the current fact's context
        for dim_qname, dim in fact.context.qnameDims.items():
            print(f"- Dimension QName: {dim_qname}")

            # Check if this dimension matches the target dimension QName
            if dim_qname.localName == target_dimension_qname:
                print(f"\nMatched Dimension: {dim.qname}")

                # For explicit dimensions, print member details
                if dim.isExplicit:
                    member_obj = dim.member
                    print(f"Member Object: {member_obj.qname}")
                    print(f"Member Properties:")
                    print(f"- Label: {member_obj.label()}")
                    print(f"- ID: {member_obj.id}")

                # For typed dimensions, print the typed member
                elif dim.isTyped:
                    typed_member = dim.typedMember
                    print(f"Typed Member Object: {typed_member}")

    # Optional: Remove or comment out the break to process all matching facts
    # break


### Adding dimensions and members to facts

In [ ]:
from arelle.ModelInstanceObject import ModelFact

def _set_dimensions(model_fact):
    """Set and print dimensions and members for each dimension separately."""
    if hasattr(model_fact.context, 'qnameDims') and model_fact.context.qnameDims:
        print(f"Context ID: {model_fact.context.id}")
        for dim_qname, dim_value in model_fact.context.qnameDims.items():
            dimension = str(dim_qname)
            if dim_value.isExplicit:
                member = str(dim_value.memberQname)
            elif dim_value.isTyped:
                member = dim_value.typedMember.stringValue if dim_value.typedMember else "TypedMember: Empty"
            else:
                member = "Unknown"
                
            print(f" - Dimension: {dimension}")
            print(f"   Member: {member}")
    else:
        print("No dimensions found in this context.")

# Iterate through all facts and process dimensions
for fact in report.model_xbrl.factsInInstance:
    _set_dimensions(fact)


In [ ]:
report.networks[4].hypercubes[0].dimensions[0].members

In [ ]:
for fact in report.facts:
    if fact.dims_members:
        print(fact.u_id, fact.dims_members)


In [ ]:
from arelle.ModelInstanceObject import ModelFact


def _set_dimensions(model_fact):
    dims_members = []
    """Set and print dimensions and members for each dimension separately."""
    if hasattr(model_fact.context, 'qnameDims') and model_fact.context.qnameDims:
        print(f"Context ID: {model_fact.context.id}")
        for dim_qname, dim_value in model_fact.context.qnameDims.items():
            dims_members.append((str(dim_qname), 
                    str(dim_value.memberQname) if dim_value.isExplicit
                    else dim_value.typedMember.stringValue if dim_value.isTyped
                    else "Unknown"
                ))
                
        print(f" - dims_members: {dims_members}")
    else:
        print("No dimensions found in this context.")

# Iterate through all facts and process dimensions
for fact in report.model_xbrl.factsInInstance:
    _set_dimensions(fact)


In [ ]:
from arelle.ModelInstanceObject import ModelFact

def _set_dimensions(model_fact):
    dims_members = []
    """Set and print dimensions and members for each dimension separately."""
    if hasattr(model_fact.context, 'qnameDims') and model_fact.context.qnameDims:
        print(f"Context ID: {model_fact.context.id}")
        for dim_qname, dim_value in model_fact.context.qnameDims.items():
            dims_members.append((str(dim_qname), 
                    str(dim_value.memberQname) if dim_value.isExplicit
                    else dim_value.typedMember.stringValue if dim_value.isTyped
                    else "Unknown"
                ))
                
        print(f" - dims_members: {dims_members}")
    else:
        print("No dimensions found in this context.")

# Iterate through all facts and process dimensions
for fact in report.model_xbrl.factsInInstance:
    _set_dimensions(fact)


In [ ]:
dir(report.facts[4].model_fact.context)

 - Dimension: us-gaap:DisposalGroupClassificationAxis
   Member: us-gaap:DiscontinuedOperationsHeldforsaleMember
 - Dimension: us-gaap:IncomeStatementBalanceSheetAndAdditionalDisclosuresByDisposalGroupsIncludingDiscontinuedOperationsAxis
   Member: alco:AlicoCitrusMember

In [54]:
for dim_concept, dim_value in report.facts[4].model_fact.context.segDimValues.items():
    print(f"Dimension: {dim_concept}, Member: {dim_value.member}")



In [ ]:
report.facts[2].model_fact.context.qnameDims

In [ ]:
report.networks[4].hypercubes[0].dimensions[0].members[0].member


In [ ]:
report.facts[2].model_fact.context.segDimValues.items()


In [ ]:
dims_members = []
def _set_dimensions(model_fact):
    """Set dimensions and members using ModelConcept objects for explicit dims
    and typed values for typed dims"""
    # Check both segDimValues and scenDimValues as per XBRL spec
    dim_values = getattr(model_fact.context, 'segDimValues', {})
    print(dim_values)
    dim_values.update(getattr(model_fact.context, 'scenDimValues', {}))
    
    if not dim_values:
        print(f"No dimensions found for fact {model_fact.qname}")
        return
        
    for dim_concept, dim_value in dim_values.items():
        if not dim_concept or not dim_value:
            print(f"Skipping dimension {dim_concept} due to missing values")
            continue
            
        try:
            # For explicit dimensions, use the member ModelConcept
            if dim_value.isExplicit and dim_value.member:
                member = dim_value.member
            # For typed dimensions, use the typed value
            elif dim_value.isTyped and dim_value.typedMember:
                member = dim_value.typedMember.stringValue
            else:
                print(f"Neither explicit nor typed for {dim_concept}")
                continue  # Skip if neither explicit nor typed
                
            dims_members.append((dim_concept, member))
            print(f" - dims_members: {dims_members}")
        except AttributeError as e:
            print(f"Warning: Could not process dimension value for {dim_concept}: {e}")

_set_dimensions(report.facts[3].model_fact)

In [ ]:
for fact in report.facts:

In [60]:
for k, v in report.facts[7].model_fact.context.segDimValues.items():
    print(f"Dimension: {k.qname}, Member: {v.member.qname}")


In [61]:
for k,v in zip(list(report.facts[3].model_fact.context.segDimValues.keys()), list(report.facts[2].model_fact.context.segDimValues.values())):
    print(f"Dimension: {k.qname}, Member: {v.member.qname}")



In [ ]:
list(report.facts[3].model_fact.context.segDimValues.keys())[1]

In [63]:
# list(report.facts[3].model_fact.context.segDimValues.values())[0].qname
# list(report.facts[2].model_fact.context.segDimValues.values())[0].member

In [ ]:
list((report.facts[2].model_fact.context.qnameDims.values()))[0].member


In [ ]:
for fact in report.model_xbrl.factsInInstance:
    for dim_qname, dim_value in fact.context.qnameDims.items():
        print(dim_qname, dim_value)


In [ ]:
from arelle.ModelInstanceObject import ModelFact


def _set_dimensions(model_fact):
    dims_members = []
    """Set and print dimensions and members for each dimension separately."""
    if hasattr(model_fact.context, 'qnameDims') and model_fact.context.qnameDims:
        print(f"Context ID: {model_fact.context.id}")
        for dim_qname, dim_value in model_fact.context.qnameDims.items():
            dims_members.append((str(dim_qname), 
                    str(dim_value) if dim_value.isExplicit
                    else dim_value.typedMember.stringValue if dim_value.isTyped
                    else "Unknown"
                ))
                
        print(f" - dims_members: {dims_members}")
    else:
        print("No dimensions found in this context.")

# Iterate through all facts and process dimensions
for fact in report.model_xbrl.factsInInstance:
    _set_dimensions(fact)


In [ ]:
# Print full details of dimensions and members for each fact
for fact in report.model_xbrl.factsInInstance:
    print(f"\nFact: {fact}")
    
    # Get dimensions from context
    if hasattr(fact.context, "qnameDims"):
        for dim_qname, dim in fact.context.qnameDims.items():
            print(f"\nDimension Object Qname: {dim.qname}")  # The full dimension object
            
            # For explicit dimensions, get member object
            if dim.isExplicit:
                member_obj = dim.member  # Get the actual member object
                print(f"Member Object: {member_obj.qname}")
                print(f"Member Properties:")
                print(f"- Label: {member_obj.label()}")
                print(f"- ID: {member_obj.id}")
                # Print other member properties you need
            
            # For typed dimensions
            elif dim.isTyped:
                typed_member = dim.typedMember
                print(f"Typed Member Object: {typed_member}")

    # Remove break to see all facts
    # break


### Member Hierarchy

In [ ]:
def display_hierarchy(dim):
    hierarchy = dim.member_hierarchy

    def print_hierarchy(parent, level=0):
        children = hierarchy.get(parent, [])
        for child in children:
            print("  " * level + f"- {child}")
            print_hierarchy(child, level + 1)

    # Start from top-level members (parent is None)
    print(f"Dimension QName: {dim.qname}")
    print_hierarchy(None)

# Iterate over all networks and hypercubes
for network in report.networks:
    if not network.hypercubes:
        continue

    print("\nNetwork URI:", network.network_uri)

    for hypercube in network.hypercubes:
        if not hypercube.dimensions:
            continue

        for dim in hypercube.dimensions:
            if hasattr(dim, 'member_hierarchy'):
                display_hierarchy(dim)


In [ ]:
import pprint

network_uri = "http://www.alicoinc.com/role/SummaryofSignificantAccountingPoliciesDisaggregationofrevenueDetails"

def print_member_hierarchy(hierarchy, parent=None, indent=0):
    children = hierarchy.get(parent, [])
    for child in children:
        print("  " * indent + f"- {child}")
        print_member_hierarchy(hierarchy, child, indent + 1)

network_uri = "http://www.alicoinc.com/role/SummaryofSignificantAccountingPoliciesDisaggregationofrevenueDetails"

for network in report.networks:
    if network.network_uri == network_uri:
        for dimension in network.hypercubes[0].dimensions:
            print(f"Dimension QName: {dimension.qname}")

            
            print_member_hierarchy(dimension.member_hierarchy)


In [ ]:
report.networks[4].hypercubes[0].dimensions[0].member_hierarchy

In [ ]:
report.networks[4].hypercubes[0].dimensions[0].members_by_level

In [ ]:
dim.dimension.members_dict.values()

In [ ]:
dir(dim)

In [ ]:
type(dim)

In [ ]:
for fact in report.facts:
    if fact.dimensions:
        print(fact.u_id, fact.members)
        break



In [ ]:
dir(report.facts[0])

In [ ]:
for network in report.networks:
    print(network.network_uri, "Validation: ", len(network.validate_facts()), "\n")



In [ ]:
stop


# Network Linking

In [ ]:
sample_network = report.networks[0]
sample_network.network_uri

In [21]:
calc_rel_set = sample_network.model_xbrl.relationshipSet(XbrlConst.summationItem, sample_network.network_uri)

sample_network.report = report
sample_network.taxonomy = report.taxonomy
validated_facts = sample_network.validate_facts()

fact_lookup: Dict[str, List[Fact]] = defaultdict(list)
for fact in validated_facts:
    fact_lookup[fact.concept.u_id].append(fact)
    
for rel in calc_rel_set.modelRelationships:
    if not (rel.fromModelObject and rel.toModelObject):
        continue
        
    # Fixed ID construction to match fact concept IDs
    parent_id = f"{rel.fromModelObject.qname.namespaceURI}:us-gaap:{rel.fromModelObject.qname.localName}"
    child_id = f"{rel.toModelObject.qname.namespaceURI}:us-gaap:{rel.toModelObject.qname.localName}"
    
    parent_facts = fact_lookup.get(parent_id, [])
    child_facts = fact_lookup.get(child_id, [])
    
    for parent_fact in parent_facts:
        for child_fact in child_facts:
            if parent_fact.context_id == child_fact.context_id:
                rel_props = {
                    'network_uri': network.network_uri,
                    'network_name': network.name,
                    'weight': rel.weight,
                    'order': rel.order,
                    'company_cik': report.company.cik,
                    'report_instance': report.instanceFile
                }

In [ ]:
parent_facts

In [ ]:
for node in sample_network.presentation.nodes.values():
    print("*"*100)
    print("concept_ID:", node.concept.u_id,"Category:", node.concept.category, "children", node.children)
    # print("ReportElementClassifier:",ReportElementClassifier.classify(node.concept.model_concept).value)
     
    print("\nNode details:")    
    print(f"Node: {node}")   
    print(f"Type: {type(node)}")
    print(f"Attributes: {dir(node)}")
    print(f"Concept type: {type(node.concept)}")
    print(f"Concept attributes: {dir(node.concept)}")
    # break

In [ ]:
relationships.append((fact, taxonomy_dim, RelationType.FACT_DIMENSION))

In [ ]:
report.facts[0]

In [ ]:

data = {}

# Loop through the nodes and extract information
for node in sample_network.presentation.nodes.values():
    
    # Either parent/child is AbstractConcept or regular Concept - If regular Concept, it has a List of Facts, we need to match which of our validated fact beongs to this concept
    parent_concept = node.concept
    child_concept = node.children[0].concept
    if parent_concept.is_abstract:
        parent_uID = parent_concept.u_id
    else:
        parent_uID = parent_concept.u_id
    
    # Skip if parent_concept, then there is no Linking - TODO: Add as a Sibling or think
    if parent_concept is None:
        continue
    
    parent_label = parent_concept
    
    # If the parent is not in the data dictionary, initialize it
    if parent_label not in data:
        data[parent_label] = {
            'Children': [],
            'Order': node.order,
            'Level': node.level,
            'Abstract': parent_concept.isAbstract
        }
    
    # Append the children to the parent's list
    data[parent_label]['Children'].extend(node.children)

data

In [ ]:
data

In [ ]:
for node_id, node in sample_network.presentation.nodes.items():
    print(node_id, node)

In [ ]:

relationships = []

# Process each presentation network
for network in sample_network.networks:
    if not network.isPresentation:
        continue
        
    # network.report = self
    # network.taxonomy = self.taxonomy
    validated_facts = network.validate_facts() # returns Set[Fact]
    validated_facts
    

# Networks - validation

In [ ]:
for network in report.networks:
    if network.isPresentation:
        print(network.network_uri)
        break


In [ ]:
network.presentation.nodes

In [ ]:
node.concept.__class__ == AbstractConcept

In [ ]:
len(node.concept.facts)



In [ ]:
[concept for hypercube in network.hypercubes for concept in hypercube.concepts]


In [ ]:
for node in network.presentation.nodes.values():
    print(dir(node))
    break



In [ ]:
# Each node_value has : 'children', 'concept', 'concept_id', 'level', 'order'

set([
    fact.u_id 
    for node in network.presentation.nodes.values() 
    for fact in node.concept.facts 
    if node.concept.__class__ != AbstractConcept and node.concept not in [concept for hypercube in network.hypercubes for concept in hypercube.concepts]
])



In [ ]:
for node in network.presentation.nodes.values():
    # Each node_value has : 'children', 'concept', 'concept_id', 'level', 'order'
    print(f"Concept: {node.concept.u_id}, Level: {node.level}, Order: {node.order}")


In [ ]:
network.presentation.__dict__

In [ ]:
report.networks[4].hypercubes[0].is_all, report.networks[4].hypercubes[0].closed

In [ ]:
# Fact Validation for Presentation Network

pr_concepts_ids = set()
pr_concepts_hypercubes = {}

# Step 1: Iterate over each network in Presentation
for network in report.networks:
    if network.isPresentation:
        print(network.network_uri, network.name, network.isPresentation, "\n")

        # Step 2: Get all concepts (concept_id) in the network 
        for key, value in network.presentation.nodes.items():    
            if not value.concept.is_abstract:
                pr_concepts_ids.add(value.concept.u_id)
                # print(pr_concepts_ids)

        # Step 3: Check presence of each of these concepts in each hypercube in this network
        if network.isDefinition: # Explicit Hypercubes
            for hypercube in network.hypercubes:
                for concept in hypercube.concepts:
                    if concept.u_id in pr_concepts_ids:
                        print(concept.u_id)

        break


In [ ]:
stop

In [ ]:
stop


In [ ]:
report.networks[4].hypercubes[0]

In [ ]:
report.networks[4].hypercubes[0].abstracts

In [ ]:
report.networks[4].network_uri, report.networks[4].hypercubes[0].hypercube_item

In [ ]:
network_uri = 'http://www.alicoinc.com/role/CONSOLIDATEDSTATEMENTSOFOPERATIONS'
     = report.networks[4].hypercubes[0].hypercube_item     
print(hypercube_item)   

In [96]:

instance_url = 'https://www.sec.gov/Archives/edgar/data/3545/000000354524000128/alco-20240930_htm.xml'
from arelle import Cntlr
from arelle.ModelXbrl import load
controller = Cntlr.Cntlr(logFileName="logfile.txt")
model_xbrl = controller.modelManager.load(instance_url)
    

In [ ]:

all_set = model_xbrl.relationshipSet(XbrlConst.all, network_uri)
domain_member = model_xbrl.relationshipSet(XbrlConst.domainMember, network_uri)
print(f"Network: {network_uri}")
for rel in all_set.modelRelationships:
    print(f"From: {rel.fromModelObject.qname} -> To: {rel.toModelObject.qname}, Closed: {rel.isClosed}, Arcrole: {rel.arcrole.split('/')[-1]}")

    for member_rel in domain_member.fromModelObject(rel.fromModelObject):
            print(f"From: {member_rel.fromModelObject.qname} -> To: {member_rel.toModelObject.qname}, Closed: {member_rel.isClosed}, Arcrole: {member_rel.arcrole.split('/')[-1]}")



In [ ]:
def print_hypercube_relationships(model_xbrl, network_uri):
    # Get relationship sets
    all_set = model_xbrl.relationshipSet(XbrlConst.all, network_uri)
    domain_member = model_xbrl.relationshipSet(XbrlConst.domainMember, network_uri)
    
    print(f"\nNetwork: {network_uri}")
    
    # Process all has-hypercube relationships
    for rel in all_set.modelRelationships:
        if rel.fromModelObject is not None and rel.toModelObject is not None:
            print(f"\nFrom: {rel.fromModelObject.qname} -> To: {rel.toModelObject.qname}")
            print(f"Closed: {rel.closed}")
            
            # Get all concepts under this primary item through domain-member relationships
            def get_concepts(concept):
                concepts = []
                if concept is not None:
                    concepts.append(concept)
                    for member_rel in domain_member.fromModelObject(concept):
                        if member_rel.toModelObject is not None:
                            concepts.extend(get_concepts(member_rel.toModelObject))
                return concepts
            
            # Get and print all related concepts
            related_concepts = get_concepts(rel.fromModelObject)
            for concept in related_concepts:
                print(f"  Related Concept: {concept.qname}")

# Use the function
print_hypercube_relationships(model_xbrl, network_uri)

In [ ]:
all_set = model_xbrl.relationshipSet(XbrlConst.all)
for rel in all_set.modelRelationships:
    print(f"From: {rel.fromModelObject.qname} -> To: {rel.toModelObject.qname}, Closed: {rel.isClosed}, Arcrole: {rel.arcrole.split('/')[-1]}")


In [ ]:
len(report.networks)


In [ ]:
report.networks[0].isDefinition


In [ ]:
# Display all rows in a pandas DataFrame
pd.set_option('display.max_rows', None)
# Create DataFrame with all networks including property methods
networks_df = pd.DataFrame([
    {
        **network.__dict__,
        'isDefinition': network.isDefinition,
        'isPresentation': network.isPresentation,
        'isCalculation': network.isCalculation,
        'networkType': network.networkType
    }
    for network in report.networks
])
networks_df.head()


In [ ]:
# Value Counts for each column in a Class: Change Concept to Network, Fact, etc
for cols in networks_df.columns.difference(['model_xbrl','id', 'hypercubes', 'concepts', 'presentation', 'name']):
    print('\n',cols)
    print(networks_df[cols].value_counts())
    print(networks_df[cols].value_counts().sum())


In [ ]:
networks_df[networks_df['isPresentation'] == True][networks_df.columns.difference(['hypercubes','model_xbrl','presentation'])]['id'].tolist()



In [ ]:
for network in report.networks:
    if (len(network.hypercubes) > 0):
        for hypercube in network.hypercubes:
            if (len(hypercube.concepts) > 0):                                
                for concept in hypercube.concepts:
                    # print(len(hypercube.concepts))
                    print(f"Concept: {concept}")
                    break
                
                # print(f"{network.name}: {hypercube.hypercube_item.qname}, Concepts: {[concept.qname for concept in hypercube.concepts]}")
        

In [ ]:
report.networks[4].hypercubes[0].abstracts

In [ ]:
report.abstracts

In [ ]:
stop

In [ ]:
report.networks[4].hypercubes[0].hypercube_item


In [ ]:
for rel in report.model_xbrl.relationshipSet(XbrlConst.all).modelRelationships:
    if rel.linkrole == 'http://xbrl.sec.gov/ecd/role/PvpDisclosureWithAdditionalMeasures':
        print(rel.linkrole, rel.toModelObject.qname)


In [ ]:
pd.set_option('display.max_colwidth', None)
networks_df[networks_df['isPresentation'] == False][networks_df.columns.difference(['hypercubes'])]

In [ ]:
networks_df.networkType.value_counts()


In [ ]:
networks_df[(networks_df['isDefinition'] == True) & (networks_df['isPresentation'] == False)][networks_df.columns.difference(['hypercubes'])]

# DEFINITION NETWORKS

In [ ]:
network_uri = 'http://xbrl.sec.gov/ecd/role/TabularListMeasures'
for i,network in enumerate(report.networks):
    if network.network_uri == network_uri:
        TabularListMeasures_hypercubes = network.hypercubes
        break
TabularListMeasures_hypercubes


In [ ]:
len(TabularListMeasures_hypercubes), len(TabularListMeasures_hypercubes[0].dimensions)

In [ ]:
print(f"Dimension 1: {TabularListMeasures_hypercubes[0].dimensions[0].qname}: members: {[member.qname for member in TabularListMeasures_hypercubes[0].dimensions[0].members]}")
print(f"Dimension 2: {TabularListMeasures_hypercubes[0].dimensions[1].qname}: members: {[member.qname for member in TabularListMeasures_hypercubes[0].dimensions[1].members]}")
print(f"Dimension 3: {TabularListMeasures_hypercubes[0].dimensions[2].qname}: members: {[member.qname for member in TabularListMeasures_hypercubes[0].dimensions[2].members]}")



In [ ]:
for i,network in enumerate(report.networks):
    if network.hypercubes:
        print(i,network.name)
        break




In [ ]:
[member.qname for member in TabularListMeasures_hypercubes[0].dimensions[1].members]

In [ ]:
[member.qname for member in TabularListMeasures_hypercubes[0].dimensions[2].members]

In [ ]:
# HYPERCUBES
for i,hypercube in enumerate(report.networks[4].hypercubes):
    print(i,hypercube.name)
    break

In [ ]:
report.networks[4].__dict__

In [ ]:
report.networks[4].hypercubes[0].dimensions[0].members[1].__dict__

# Presentation Network

In [ ]:
sample_network_idx = 0
sample_network = report.networks[sample_network_idx]
len([concept for hypercube in sample_network.hypercubes for concept in hypercube.concepts])

In [18]:

def validate_facts(sample_network) -> List[Fact]:

    # Step 1: Get all facts (without dimensions) for concepts in this presentation that are not in any hypercubes of this network
    fact_list = set([
        fact
        for node in getattr(sample_network.presentation, 'nodes', {}).values()  # Safely get nodes if presentation exists - self.presentation.nodes.values()
        if node.concept and node.concept.__class__ != AbstractConcept  # Ensure concept is not None and not abstract
        and node.concept not in [concept for hypercube in sample_network.hypercubes for concept in hypercube.concepts]  # Exclude concepts in hypercubes
        for fact in node.concept.facts
        if not fact.dims_members  # Include only facts with no dimensions
    ])

    return fact_list

In [ ]:
validate_facts(sample_network)

In [ ]:
empty_member_facts = [fact for fact in report.facts if fact.dims_members and any(not member for _, member in fact.dims_members)]
empty_member_facts

In [ ]:
for fact in report.facts:
    if fact.dims_members:
        print(fact.u_id)
        break


In [ ]:
sample_network_idx = 0
sample_network = report.networks[sample_network_idx]
len([concept for hypercube in sample_network.hypercubes for concept in hypercube.concepts])

In [ ]:
report.facts[14].dims_members[0][0].members

In [ ]:
report.facts[14].dims_members[0]

In [ ]:
report.facts[14].dims_members[0][0].default_member


In [ ]:
report.taxonomy.dimensions[0].members_dict

In [ ]:
networks.hypercubes[0].dimensions[0].members_dict

In [ ]:
report.networks[4].hypercubes[0].dimensions[0].members_dict

# Presentation Network

    # Step1: Get all facts (W/O Dims)for concepts in this presentation that are not in any hypercubes of this network
    # node.concept.__class__ != AbstractConcept

    # Each node_value has : 'children', 'concept', 'concept_id', 'level', 'order'


In [ ]:
sample_network = report.networks[0]
sample_network.name

In [ ]:
report.abstracts[:2]

In [15]:
# for network in report.networks:
#     if network.isPresentation:
#         for node in network.presentation.nodes.values():
#             if node.concept is not None:    
#                 parent_concept = node.concept.model_concept
#                 if parent_concept is not None:
#                     if str(parent_concept.qname) not in abstract_concepts:
#                         print(str(parent_concept.qname))

In [ ]:
report.report

In [ ]:
node.children

In [ ]:
for node in sample_network.presentation.nodes.values():
      for child in node.children:
          parent_concept = node.concept.model_concept
          child_node = sample_network.presentation.nodes[child]
          print(f"Parent: {parent_concept.qname},child_uid: {child_node.concept.u_id} Children: {child}, Order: {node.order}, Level: {node.level}, Abstract: {parent_concept.isAbstract}")
    #   break

            

In [ ]:
for node in sample_network.presentation.nodes.values():
    parent_concept = node.concept.model_concept
    if parent_concept is not None:  # Check if parent_concept is not None
        num_children = len(node.children)  # Count the number of children
        print(f"Parent: {parent_concept.qname}, Number of Children: {num_children}, Order: {node.order}, Level: {node.level}, Abstract: {parent_concept.isAbstract}")
        for child in node.children:
            print(f"    Child: {child}")
    else:
        print(f"Parent: None, Number of Children: {len(node.children)}, Order: {node.order}, Level: {node.level}")


In [14]:

# Display all rows and columns without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
import pandas as pd

# Initialize a dictionary to store data for the DataFrame
data = {}

# Loop through the nodes and extract information
for node in sample_network.presentation.nodes.values():
    parent_concept = node.concept.model_concept
    
    # Skip if parent_concept is None
    if parent_concept is None:
        continue
    
    parent_label = parent_concept.label()
    
    # If the parent is not in the data dictionary, initialize it
    if parent_label not in data:
        data[parent_label] = {
            'Children': [],
            'Order': node.order,
            'Level': node.level,
            'Abstract': parent_concept.isAbstract
        }
    
    # Append the children to the parent's list
    data[parent_label]['Children'].extend(node.children)

# Convert the dictionary into a DataFrame
df = pd.DataFrame([
    {'Parent': parent, 'Children': children['Children'], 'Order': children['Order'], 'Level': children['Level'], 'Abstract': children['Abstract']}
    for parent, children in data.items()
])

df

In [32]:
# import pandas as pd

# # Initialize a list to store each node's data as a dictionary
# data = []

# # Iterate over all concept-node pairs in the presentation network
# for concept, node in sample_network.presentation.nodes.items():
#     node_data = {
#         'ParentID': concept,
#         'order': node.order,
#         'level': node.level,
#         'children': ', '.join(node.children) if node.children else None
#     }
#     data.append(node_data)

# # Create a pandas DataFrame from the list of dictionaries
# df = pd.DataFrame(data)

# # Optional: Display the DataFrame
# df


In [ ]:
node = sample_network.presentation.nodes.get('http://fasb.org/us-gaap/2024:us-gaap:StatementOfCashFlowsAbstract')
node.concept

In [ ]:
report.facts[10].concept.u_id


# STOP

In [ ]:
network_id = 2
print(f"Presentation Network: {report.networks[network_id].network_uri}")
report.networks[network_id].presentation.nodes

In [ ]:
report.networks[network_id].presentation.roots

In [ ]:
node = report.networks[network_id].presentation.nodes['http://fasb.org/us-gaap/2023:us-gaap:StatementOfFinancialPositionAbstract']
# Access both structural info and concept instance
print(f"Level: {node.level}")  # Hierarchy level
print(f"Order: {node.order}")  # Order within level
print(f"Concept: {node.concept}")  # Actual concept instance
print(f"Children: {report.networks[network_id].presentation.get_children(node)}")

In [ ]:
definition, presentation, total = 0, 0, 0
for network in report.networks:
    total += 1
    if network.isDefinition:
        definition += 1
    elif network.isPresentation:
        presentation += 1

print(f"Definition Networks: {definition}\nPresentation Networks: {presentation}\nTotal Networks: {total}")
    


In [ ]:
sum(1 for network in report.networks if network.presentation)

In [ ]:
report.networks[2].presentation.__dict__

In [124]:
instance_url = 'https://www.sec.gov/Archives/edgar/data/3545/000000354524000128/alco-20240930_htm.xml'
from arelle import Cntlr
from arelle.ModelXbrl import load
controller = Cntlr.Cntlr(logFileName="logfile.txt")
model_xbrl = controller.modelManager.load(instance_url)

In [ ]:
unique_roles = {}
for i, role in enumerate(model_xbrl.roleTypes):
    unique_roles[i] = role.split(':')[1]
    print(i,role)

len(unique_roles)

# Presentation Hierarchy

In [29]:
instance_url = 'https://www.sec.gov/Archives/edgar/data/3545/000000354524000128/alco-20240930_htm.xml'
instance_url = 'https://www.sec.gov/Archives/edgar/data/1784970/000121390024106993/ea0223953-10qa1_nexttech_htm.xml' # 10-Q Amended report
from arelle import Cntlr
from arelle.ModelXbrl import load
controller = Cntlr.Cntlr(logFileName="logfile.txt")
model_xbrl = controller.modelManager.load(instance_url)


In [ ]:
from arelle import XbrlConst

def get_calculation_network_children_ordered(model_xbrl, network_linkrole):
    relationship_set = model_xbrl.relationshipSet(XbrlConst.summationItem, network_linkrole)
    if not relationship_set:
        return []

    ordered_rels = []
    
    # Iterate through all parent concepts
    for parent in relationship_set.fromModelObjects():
        # For each parent, get its child relationships
        for rel in relationship_set.fromModelObject(parent):
            # Append parent-child relationships with order
            ordered_rels.append({
                'parent': parent.qname,
                'child': rel.toModelObject.qname,
                'order': rel.order
            })

    return ordered_rels

network_name = 'Condensed Consolidated Balance Sheets (Unaudited)'.lower()
matching_roles = [(uri, rt.definition) for uri, rts in model_xbrl.roleTypes.items() for rt in rts if rt.definition and network_name in rt.definition.lower()]
for uri, definition in matching_roles: print(f"Role URI: {uri}\nDefinition: {definition}\n")

network_linkrole = next((uri for uri, rts in model_xbrl.roleTypes.items() for rt in rts if rt.definition and network_name in rt.definition.lower()), None)
print(network_linkrole, network_name)
ordered_rels = get_calculation_network_children_ordered(model_xbrl, network_linkrole)
ordered_rels


In [32]:
from arelle import XbrlConst

def get_network_children_ordered(model_xbrl, network_linkrole):
    relationship_set = model_xbrl.relationshipSet(XbrlConst.parentChild, network_linkrole)
    if not relationship_set:
        return []

    ordered_rels = []
    
    # Iterate through all parent concepts
    for parent in relationship_set.fromModelObjects():
        # For each parent, get its child relationships
        for rel in relationship_set.fromModelObject(parent):
            # Append parent-child relationships with order
            ordered_rels.append({
                'parent': parent.qname,
                'child': rel.toModelObject.qname,
                'order': rel.order
            })

    return ordered_rels


In [ ]:
network_name = 'Condensed Consolidated Balance Sheets (Unaudited)'.lower()
matching_roles = [(uri, rt.definition) for uri, rts in model_xbrl.roleTypes.items() for rt in rts if rt.definition and network_name in rt.definition.lower()]
for uri, definition in matching_roles: print(f"Role URI: {uri}\nDefinition: {definition}\n")


In [ ]:
network_linkrole = next((uri for uri, rts in model_xbrl.roleTypes.items() for rt in rts if rt.definition and network_name in rt.definition.lower()), None)
print(network_linkrole, network_name)
ordered_rels = get_network_children_ordered(model_xbrl, network_linkrole)
ordered_rels


In [ ]:
def get_presentation_network_structure(model_xbrl, network_name):
    """
    Gets complete presentation network structure using hierarchical ordering.
    """
    # network_linkrole = next(role for role in model_xbrl.roleTypes if network_name in role)
    network_linkrole = next((uri for uri, rts in model_xbrl.roleTypes.items() for rt in rts if rt.definition and network_name in rt.definition.lower()), None)
    
    # Get ordered relationships first
    ordered_rels = get_network_children_ordered(model_xbrl, network_linkrole)
    
    # Group relationships by parent
    hierarchy_by_parent = {}
    all_parents = set()
    all_children = set()
    
    for rel in ordered_rels:
        parent = str(rel['parent'])
        child = str(rel['child'])
        if parent not in hierarchy_by_parent:
            hierarchy_by_parent[parent] = []
        hierarchy_by_parent[parent].append(rel)
        all_parents.add(parent)
        all_children.add(child)
    
    # Find root(s) - parents that are not children of any other concept
    root_concepts = all_parents - all_children
    
    structure = []
    processed = set()
    level_orders = {}  # Track the order per level
    
    def process_parent(parent_concept, level):
        if parent_concept not in hierarchy_by_parent:
            return
        
        if level not in level_orders:
            level_orders[level] = 1  # Start order from 1 instead of 0
        
        children = sorted(hierarchy_by_parent[parent_concept], key=lambda x: x['order'])
        
        for rel in children:
            child_concept = str(rel['child'])
            if child_concept in processed:
                continue
                
            child_obj = model_xbrl.qnameConcepts[rel['child']]
            structure.append({
                'concept': child_concept,
                'label': child_obj.label(),
                'order': level_orders[level],  # Use level-specific order
                'level': level,
                'is_abstract': child_obj.isAbstract
            })
            level_orders[level] += 1  # Increment the level-specific order
            
            processed.add(child_concept)
            process_parent(child_concept, level + 1)
    
    # Process each root concept
    for root in root_concepts:
        if root not in processed:
            if 1 not in level_orders:
                level_orders[1] = 1  # Start root order from 1
            
            root_obj = model_xbrl.qnameConcepts[hierarchy_by_parent[root][0]['parent']]
            structure.append({
                'concept': root,
                'label': root_obj.label(),
                'order': level_orders[1],
                'level': 1,  # Start root level from 1
                'is_abstract': True
            })
            level_orders[1] += 1  # Increment the root level order
            processed.add(root)
            process_parent(root, 2)  # Start child level from 2
    
    return pd.DataFrame(structure)

# Usage
# "Axis", "Abstract", "Table", "LineItems"
print(network_name)
presentation_structure = get_presentation_network_structure(model_xbrl, network_name)
presentation_structure.sort_values(by=['level', 'order'], ascending=[True, True])


In [ ]:
import pandas as pd

def flatten_presentation_structure(df: pd.DataFrame) -> pd.DataFrame:
    # Keywords to skip
    SKIP_KEYWORDS = ("Axis", "Abstract", "Table", "LineItems")
    
    # Convert df to a parent->children mapping using the original order and level
    # We'll reconstruct the hierarchy using the concept's current level and order as guides.
    # First, find potential roots (lowest level = 1 concepts)
    # NOTE: The provided df likely already sorted by [level, order], we rely on that.
    df = df.sort_values(by=["level", "order"]).reset_index(drop=True)
    
    # Build a tree from the dataframe.
    # Concept at level N+1 that comes after a concept at level N is considered a child if:
    # - It's the next in sequence going down levels.
    # We'll do this by tracking a stack of last-seen nodes at each level.
    level_stack = []
    parent_map = {}
    
    for _, row in df.iterrows():
        concept = row['concept']
        level = row['level']
        # Ensure level_stack is trimmed to (level-1)
        while len(level_stack) >= level:
            level_stack.pop()
        if level_stack:
            parent = level_stack[-1]
            parent_map.setdefault(parent, []).append(concept)
        else:
            # No parent, this is a root
            parent_map.setdefault(None, []).append(concept)
        level_stack.append(concept)
    
    # Create a lookup for concept info
    concept_info = df.set_index('concept').to_dict('index')
    
    def is_skippable(c):
        info = concept_info[c]
        if info['is_abstract']:
            return True
        return any(c.endswith(kw) for kw in SKIP_KEYWORDS)
    
    # We'll do a DFS from the roots, and flatten by skipping abstract/axis/table/lineItems.
    flattened = []
    new_level_order = {}
    
    def ensure_level_order(lvl):
        if lvl not in new_level_order:
            new_level_order[lvl] = 1
    
    def add_node(c, lvl):
        ensure_level_order(lvl)
        flattened.append({
            'concept': c,
            'label': concept_info[c]['label'],
            'order': new_level_order[lvl],
            'level': lvl,
            'is_abstract': concept_info[c]['is_abstract']
        })
        new_level_order[lvl] += 1
    
    def process_node(c, lvl, parent_skipped=False):
        """ 
        If node is skippable, don't add it, just process children at same level if parent_skipped=True, 
        else keep them at the current lvl if parent isn't skipped.
        If node is not skippable, add it and process children at lvl+1.
        """
        children = parent_map.get(c, [])
        
        if is_skippable(c):
            # Skip this node, just promote children
            for child in children:
                process_node(child, lvl, parent_skipped=True)
        else:
            # Valid node
            add_node(c, lvl)
            for child in children:
                process_node(child, lvl+1, parent_skipped=False)
    
    # Process all roots
    roots = parent_map.get(None, [])
    for r in roots:
        if is_skippable(r):
            for child in parent_map.get(r, []):
                process_node(child, 1, parent_skipped=True)
        else:
            add_node(r, 1)
            for child in parent_map.get(r, []):
                process_node(child, 2, parent_skipped=False)
    
    result = pd.DataFrame(flattened, columns=['concept','label','order','level','is_abstract'])
    return result.sort_values(by=['level','order']).reset_index(drop=True)


# Usage
# presentation_structure is the DataFrame you already have (the one with Axis, Abstract, etc.)
flattened_df = flatten_presentation_structure(presentation_structure)
flattened_df


In [ ]:
model_xbrl

In [ ]:
report.networks[4].__dict__

# Facts Dimensions Match Taxonomy Dimensions

In [5]:
{dim.u_id: dim for dim in report.taxonomy.dimensions}

member_lookup = {}

for dim in report.taxonomy.dimensions:
    for member_qname_str, member in dim.members_dict.items():
        member_lookup[member.u_id] = member

In [ ]:
report.facts[14]

In [ ]:
for fact in report.facts:
    if not fact.dims_members:
        continue
    else:
        for dim_concept, member_concept in fact.dims_members:
            print(dim_concept.u_id, member_concept.u_id)
        break


In [6]:

def _build_fact_dimension_relationships(self) -> List[Tuple[Neo4jNode, Neo4jNode, RelationType]]:
    """Build relationships between facts and their dimensions/members"""
    relationships = []
    dim_count = 0
    member_count = 0

    # Create lookup dictionaries using QName objects
    dim_lookup = {dim.u_id: dim for dim in report.taxonomy.dimensions}
    member_lookup = {}

    for dim in self.taxonomy.dimensions:
        for member_qname_str, member in dim.members_dict.items():
            member_lookup[member.u_id] = member 

    # Process each fact's dimensions
    for fact in self.facts:
        if not fact.dims_members:
            continue

        for dim_concept, member_concept in fact.dims_members:
            if not dim_concept:
                continue

            taxonomy_dim = dim_lookup.get(dim_concept.u_id)
            if not taxonomy_dim:
                continue

            # If we have a member, try to link through it
            if member_concept:
                member = member_lookup.get(member_concept.u_id)
                if member:
                    relationships.append((fact, member, RelationType.HAS_MEMBER))
                    member_count += 1
                    continue  # Proceed to next dimension-member pair

            # If no member or member not found, link directly to dimension
            relationships.append((fact, taxonomy_dim, RelationType.HAS_DIMENSION))
            dim_count += 1

    print(f"\nBuilt fact relationships: {dim_count} direct dimensions, {member_count} members")
    print(f"Facts with dimension relationships: {len(set(r[0].u_id for r in relationships))}")
    return relationships


# Taxonomy Dimensions

In [1]:
instance_url = 'https://www.sec.gov/Archives/edgar/data/3545/000000354524000128/alco-20240930_htm.xml'
from arelle import Cntlr
from arelle.ModelXbrl import load
controller = Cntlr.Cntlr(logFileName="logfile.txt")
model_xbrl = controller.modelManager.load(instance_url)


In [ ]:
report.networks[4].hypercubes[0].dimensions[0]

In [ ]:
target_dim = "srt:FederalHomeLoanBankAdvancesBranchOfFHLBBankAxis"
target_dim = "ecd:ExecutiveCategoryAxis"

for i, network in enumerate(report.networks):
    for j, hypercube in enumerate(network.hypercubes):
        for k, dimension in enumerate(hypercube.dimensions):
            if str(dimension.qname) == target_dim:
                print(f"Found at: report.networks[{i}].hypercubes[{j}].dimensions[{k}]")
                print(f"Dimension: {dimension.qname}, {report.networks[i].name}")
                break


In [ ]:
for dim in report.taxonomy.dimensions:
    print(dim.qname)
    print(dim.item.isExplicitDimension)
    print(dim.members_dict)
    break


In [3]:
# print(len([dim.qname for dim in report.taxonomy.dimensions]))
# dim_names = [dim.qname for dim in report.taxonomy.dimensions]
# axis_count = sum(1 for dim_name in dim_names if dim_name.split(':')[-1].endswith('Axis'))
# print(f"Number of dimensions ending with 'Axis': {axis_count} out of {len(dim_names)} total dimensions")
# [dim_name.split(':')[-1] for dim_name in dim_names]



In [ ]:
from arelle import XbrlConst

i = 1
unique_dimensions = set()
unique_domains = {}
dim_dom_rel_set = model_xbrl.relationshipSet(XbrlConst.dimensionDomain)
for concept in model_xbrl.qnameConcepts.values(): # key is the concept.qname
    if concept.isDimensionItem: 
        unique_dimensions.add(concept.qname)

unique_dimensions

In [ ]:
from arelle import XbrlConst

i = 1
unique_dimensions = set()
unique_domains = {}
dim_dom_rel_set = model_xbrl.relationshipSet(XbrlConst.dimensionDomain)
for concept in model_xbrl.qnameConcepts.values(): # key is the concept.qname
    if concept.isDimensionItem: 
        unique_dimensions.add(concept.qname)
        # print(i,concept.qname)
        # i += 1
        if concept.qname not in unique_domains:
            unique_domains[concept.qname] = set()
        
        for rel in dim_dom_rel_set.fromModelObject(concept):
            
            if rel.toModelObject is not None:
                # print(rel.toModelObject.qname)
                # print(concept.qname, rel.toModelObject.qname)
                unique_domains[concept.qname].add(rel.toModelObject)


# print(i)
print("Unique Dimensions: ",len(unique_dimensions))
print("Unique Domains: ",len(unique_domains))

In [ ]:
def is_domain_only(concept):
    """Debug function to check domain characteristics"""
    # Store conditions separately
    condition1 = concept.isItem
    condition2 = 'domainItemType' in str(concept.typeQname)
    condition3 = concept.isAbstract
    
    # Check relationships in base set (without specific network)
    dim_dom_rel_set = model_xbrl.relationshipSet(XbrlConst.dimensionDomain)
    rels_to = []
    if dim_dom_rel_set:
        rels_to = list(dim_dom_rel_set.toModelObject(concept))
    condition4 = bool(rels_to)
    
    # Print debug info only if it fails
    if not all([condition1, condition2, condition3, condition4]):
        print(f"\nDEBUG for failed domain {concept.qname}:")
        print(f"1. isItem: {condition1}")
        print(f"2. has domainItemType: {condition2}")
        print(f"3. isAbstract: {condition3}")
        print(f"4. is target of dimension-domain: {condition4}")
        
        if rels_to:
            print("\nFound Relationships:")
            for rel in rels_to:
                print(f"- From: {rel.fromModelObject.qname}")
                print(f"  To: {rel.toModelObject.qname}")
                print(f"  Role: {rel.linkrole}")

    if all([condition1, condition2, condition3, condition4]):
        print(f"{concept.qname} - ISDOMAIN: True")
    else:
        print(f"{concept.qname} - ISDOMAIN: False")
    
    return all([condition1, condition2, condition3, condition4])

# Test all domains
print("Checking all domains:")
i = 1
for k, v in unique_domains.items():
    if v:  # if the set is not empty
        print(f"\n{i} Dimension: {k}")

        i += 1
        for domain in v:
            result = is_domain_only(domain)
            # print(f"{domain.qname} - ISDOMAIN: {result}")

In [ ]:
for concept in model_xbrl.qnameConcepts.values(): # key is the concept.qname
    
    print(concept.qname,  is_domain_only(concept))


In [ ]:
for k,v in unique_domains.items():
    if len(v) > 0:
        # print(k,v)
        for concept in v:
            print(concept.qname, is_domain_only(concept))


# NULL FACTS

In [13]:
instance_url = 'https://www.sec.gov/Archives/edgar/data/1784970/000121390024106993/ea0223953-10qa1_nexttech_htm.xml'
from arelle import Cntlr
from arelle.ModelXbrl import load
controller = Cntlr.Cntlr(logFileName="logfile.txt")
model_xbrl = controller.modelManager.load(instance_url)


In [ ]:
i = 0
for fact in report.facts:
    if fact.value is None:
        # print(fact.qname, fact.value, fact.id)
        print(i, fact.qname)
        i += 1


In [26]:
# actual_nil_facts = set()
# for fact in model_xbrl.facts:
#     if not fact.isNil:
#         actual_nil_facts.add(fact)
#         print(fact.qname, fact.value, fact.id)

In [ ]:
model_xbrl

In [ ]:
actual_nil_facts = set()
for fact in model_xbrl.facts:
    if fact.isNil:
        actual_nil_facts.add(fact)
        # print(fact.qname, fact.value, fact.id)
    # if fact.value is None:
    #     print(fact.qname)

for i, fact in enumerate(actual_nil_facts):
    print(i, fact.qname, fact.value, fact.id)
    


In [ ]:
report.facts[0].model_fact.value

# Categories


In [15]:
instance_url = 'https://www.sec.gov/Archives/edgar/data/1784970/000121390024106993/ea0223953-10qa1_nexttech_htm.xml'
from arelle import Cntlr
from arelle.ModelXbrl import load
controller = Cntlr.Cntlr(logFileName="logfile.txt")
model_xbrl = controller.modelManager.load(instance_url)


In [21]:
# model_xbrl.nameConcepts

In [ ]:
report.abstracts

In [ ]:
for abstract in report._abstract_lookup.values():
    print(abstract.category)

In [25]:


class ReportElementClassifier:
    LINKBASE_ELEMENTS = [
        'link:part', 'xl:extended', 'xl:arc', 'xl:simple', 
        'xl:resource', 'xl:documentation', 'xl:locator', 
        'link:linkbase', 'link:definition', 'link:usedOn', 
        'link:roleType', 'link:arcroleType'
    ]
    
    DATE_ELEMENTS = [
        'xbrli:instant', 'xbrli:startDate', 'xbrli:endDate', 
        'dei:eventDateTime', 'xbrli:forever'
    ]

    @staticmethod
    def get_substitution_group(concept: Any) -> str:
        return (str(concept.substitutionGroupQname) 
                if concept.substitutionGroupQname else "N/A")

    @staticmethod
    def get_local_type(concept: Any) -> str:
        return (str(concept.typeQname.localName) 
                if concept.typeQname else '')

    @staticmethod
    def check_nillable(concept: Any) -> bool:
        return (concept.nillable == 'true')

    @staticmethod
    def check_duration(concept: Any) -> bool:
        return (concept.periodType == 'duration')

    @classmethod
    def classify(cls, concept: Any) -> NodeType:
        qname_str = str(concept.qname)
        sub_group = cls.get_substitution_group(concept)
        local_type = cls.get_local_type(concept)
        nillable = cls.check_nillable(concept)
        duration = cls.check_duration(concept)

        # Initial classification
        category = cls._initial_classify(concept, qname_str, sub_group, 
                                      local_type, nillable, duration)
        
        # Integrated post-classification
        if category == NodeType.OTHER:
            category = cls._post_classify_single(concept, qname_str, sub_group)
        
        return category

    @classmethod
    def _initial_classify(cls, concept, qname_str, sub_group, local_type, 
                         nillable, duration) -> NodeType:
        # 1. Basic Concept
        if not concept.isAbstract and sub_group == 'xbrli:item':
            return NodeType.CONCEPT
        
        # 2. Hypercube [Table]
        if (concept.isAbstract and 
            sub_group == 'xbrldt:hypercubeItem' and
            duration and nillable):
            return NodeType.HYPERCUBE
        
        # 3. Dimension [Axis] - here we could also just use "concept.isDimensionItem"
        if (concept.isAbstract and 
            sub_group == 'xbrldt:dimensionItem' and 
            duration and nillable):
            return NodeType.DIMENSION
        
        # 4. Member [Domain/Member]
        if ((any(qname_str.endswith(suffix) for suffix in ["Domain", "domain", "Member"]) or 
             local_type == 'domainItemType') and 
            duration and nillable):
            return NodeType.MEMBER
        
        # 5. Abstract
        if any(qname_str.endswith(suffix) for suffix in [
            "Abstract", "Hierarchy", "RollUp", 
            "RollForward", "Rollforward"
        ]) and concept.isAbstract:
            return NodeType.ABSTRACT
        
        # 6. LineItems
        if "LineItems" in qname_str and duration and nillable:
            return NodeType.LINE_ITEMS
        
        # 7. Guidance
        if local_type == 'guidanceItemType' or "guidance" in qname_str.lower():
            return NodeType.GUIDANCE
        
        # 8. Deprecated
        if "deprecated" in qname_str.lower() or "deprecated" in local_type.lower():
            return NodeType.DEPRECATED

        return NodeType.OTHER

    @classmethod
    def _post_classify_single(cls, concept, qname_str, sub_group) -> NodeType:
        # Rule 1: IsDomainMember -> *Member
        if concept.isDomainMember:
            return NodeType.MEMBER
            
        # Rule 2: Abstract -> *Abstract
        if concept.isAbstract:
            return NodeType.ABSTRACT
            
        # Rule 3: LinkbaseElement SubstitutionGroups
        if sub_group in cls.LINKBASE_ELEMENTS:
            return NodeType.LINKBASE
            
        # Rule 4: Date Elements
        if qname_str in cls.DATE_ELEMENTS:
            return NodeType.DATE
            
        return NodeType.OTHER

# endregion


In [ ]:
# print(len(model_xbrl.qnameConcepts))
# for qname, concept in model_xbrl.qnameConcepts.items():
#     category = ReportElementClassifier.classify(concept).value
#     print(qname, category)
# # Get value counts of categories
# from collections import Counter

categories = Counter()
for qname, concept in model_xbrl.qnameConcepts.items():
    category = ReportElementClassifier.classify(concept).value
    categories[category] += 1

print("\nCategory Value Counts:")
print("-" * 30)
for category, count in sorted(categories.items()):
    print(f"{category}: {count}")


In [ ]:
report.abstracts

In [ ]:
for concept in report.abstracts:
    print(concept.qname, concept.category)


In [ ]:
for concept in model_xbrl.concepts:
    print(concept.qname, concept.category)


# Test Local

In [15]:
# instance_url = 'https://www.sec.gov/Archives/edgar/data/3545/000000354524000128/alco-20240930_htm.xml'
# from arelle import Cntlr
# from arelle.ModelXbrl import load
# controller = Cntlr.Cntlr(logFileName="logfile.txt")
# model_xbrl = controller.modelManager.load(instance_url)


# References

In [36]:
# Get All available information in XBRL
# report.model_xbrl.__dict__

In [37]:
# Value Counts for each column in a Class: Change Concept to Network, Fact, etc
# report_concepts = pd.DataFrame(report.concepts)
# for cols in report_concepts.columns.difference(['model_concept','concept_type']):
#     print('\n',cols)
#     print(report_concepts[cols].value_counts())
#     print(report_concepts[cols].value_counts().sum())

In [38]:
# All unique concepts associated with facts in this instance
# len(set([fact.concept for fact in report.model_xbrl.factsInInstance]))

In [39]:
# # Build Units

# import pandas as pd

# # Normalize Unit ID
# def _normalize_unit_id(unit_id):
#     if not unit_id:
#         return None
#     if isinstance(unit_id, str) and unit_id.startswith("u-"):
#         return unit_id
#     if hasattr(unit_id, "id"):
#         return f"u-{unit_id.id}"
#     return f"u-{abs(hash(str(unit_id))) % 10000}"

# # Extract only the required details
# def extract_final_facts(model_xbrl):
#     data = []
#     for model_fact in model_xbrl.factsInInstance:
#         # Extract unit-related details
#         is_divide = getattr(model_fact.unit, "isDivide", None)
#         string_value = getattr(model_fact.unit, "stringValue", None)
#         unit_ref = _normalize_unit_id(model_fact.unitID)

#         # Process UTR entries
#         utr_1 = next(iter(model_fact.utrEntries), None) if model_fact.utrEntries else None
#         utr_1_id = getattr(utr_1, "id", None)
#         utr_1_is_simple = getattr(utr_1, "isSimple", None)
#         utr_1_item_type = getattr(utr_1, "itemType", None)
#         utr_1_ns_unit = getattr(utr_1, "nsUnit", None)
#         utr_1_status = getattr(utr_1, "status", None)

#         # Add details to the data list
#         data.append({
#             "IsDivide": is_divide,
#             "StringValue": string_value,
#             "UTR_1_id": utr_1_id,
#             "UTR_1_isSimple": utr_1_is_simple,
#             "UTR_1_itemType": utr_1_item_type,
#             "UTR_1_nsUnit": utr_1_ns_unit,
#             "UTR_1_status": utr_1_status,
#             "UnitRef": unit_ref,
#         })

#     return pd.DataFrame(data)

# # Assuming model_xbrl is defined elsewhere in the environment
# unit_df = extract_final_facts(model_xbrl)
# display(unit_df.head())


# Contexts


In [42]:
instance_url = 'https://www.sec.gov/Archives/edgar/data/3545/000000354524000128/alco-20240930_htm.xml'
instance_url = 'https://www.sec.gov/Archives/edgar/data/1784970/000121390024106993/ea0223953-10qa1_nexttech_htm.xml'
from arelle import Cntlr
from arelle.ModelXbrl import load
controller = Cntlr.Cntlr(logFileName="logfile.txt")
model_xbrl = controller.modelManager.load(instance_url)


In [ ]:
## build contexts
import pandas as pd

# Assuming model_fact.context is a complex structure
contexts = []
for model_fact in model_xbrl.factsInInstance:
    # Assuming `model_fact.context` can be converted to a dictionary or has attributes
    context_data = model_fact.context if isinstance(model_fact.context, dict) else vars(model_fact.context)
    contexts.append(context_data)

# Creating the DataFrame from a list of dictionaries
context_df = pd.DataFrame(contexts)

# Display the DataFrame
context_df


In [52]:
def get_all_contexts(model_xbrl):
    """Get all unique contexts from XBRL report"""
    contexts = set(model_xbrl.contexts.values())
    [contexts.add(fact.context) for fact in model_xbrl.facts if fact.context]
    
    for rel in [r for rs in model_xbrl.relationshipSets.values() for r in rs.modelRelationships]:
        if hasattr(rel, 'fromModelObject') and getattr(rel.fromModelObject, 'context', None):
            contexts.add(rel.fromModelObject.context)
        if hasattr(rel, 'toModelObject') and getattr(rel.toModelObject, 'context', None):
            contexts.add(rel.toModelObject.context)
    
    return list(contexts)


def analyze_contexts(model_xbrl):
    """Print all context details with proper period handling"""
    for ctx in get_all_contexts(model_xbrl):
        print(f"\nContext {ctx.id}")
        print(f"Entity: {ctx.entityIdentifier}")
        
        # Period handling using same logic as Period class
        period_type = ("instant" if getattr(ctx, "isInstantPeriod", False)
                      else "duration" if getattr(ctx, "isStartEndPeriod", False)
                      else "forever")
        
        # Extract and format dates
        if period_type == "instant":
            print(f"Period: instant at {ctx.instantDatetime.strftime('%Y-%m-%d')}")
        elif period_type == "duration":
            start = ctx.startDatetime.strftime('%Y-%m-%d')
            end = ctx.endDatetime.strftime('%Y-%m-%d')
            print(f"Period: duration from {start} to {end}")
        else:
            print("Period: forever")
            
        # Dimensions if any
        if ctx.qnameDims:
            print("Dimensions:")
            for dim, mem in ctx.qnameDims.items():
                print(f"  {dim}: {mem.memberQname if hasattr(mem, 'memberQname') else mem}")
                print(f"  {dim}: {mem.memberQname if hasattr(mem, 'memberQname') else mem}")


analyze_contexts(model_xbrl)                